# Finding ways to reduce hydrogen demand in the models 

Author: Katherine Shaw

Date: August 4 

Loading on of the minimums scenarios from the first (late july) run-throughs of the model 
reminder: one node per country, 26H resolution, all sectors, dac turned on. 


In [12]:
#Packages 
import pypsa
import matplotlib.pyplot as plt 
import cartopy 
import geopandas
import networkx
import linopy
import cartopy.crs as ccrs
import atlite 
import geopandas as gpd 
import xarray
import pandas as pd 
from datetime import datetime
import numpy as np
from pypsa.plot import add_legend_patches
import random
import plotly.graph_objects as go

## Overall view of colors matched to carriers 
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

#importing auxillary functions
import Auxillary_Functions as af

In [2]:
network_2020 = pypsa.Network('/Users/katherine.shaw/Desktop/Late_July_Hydrogen_tests_1/30%/networks/base_s_39___2020.nc')
network_2030 = pypsa.Network('/Users/katherine.shaw/Desktop/Late_July_Hydrogen_tests_1/30%/networks/base_s_39___2030.nc')
network_2040 = pypsa.Network('/Users/katherine.shaw/Desktop/Late_July_Hydrogen_tests_1/30%/networks/base_s_39___2040.nc')
network_2050 = pypsa.Network('/Users/katherine.shaw/Desktop/Late_July_Hydrogen_tests_1/30%/networks/base_s_39___2050.nc')

INFO:pypsa.io:Imported network base_s_39___2020.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_39___2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_39___2040.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_39___2050.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [3]:
#Building a Sankey diagram to see where EU oil goes, because that's where all the hydrogen demand ends up going 

#building off the Sankey_biomass_hydrogen code in auxillary functions. 
def Sankey_from_bus(network_choice, bus_choice):
    network = network_choice

    #create a dataframe for the Sankey
    Sankey_data = pd.DataFrame(columns = ['link name'])

    #the bus from with the link need to originate 
    source_bus = bus_choice 

    #identifying all the links with the bus of interest
    links = network.links[network.links.bus0 == source_bus].index.copy()
    Sankey_data['link name'] = links.values

    #listing source bus
    Sankey_data['source bus'] = Sankey_data['link name'].map((network.links.loc[links, 'bus0']))
    #identifying the target buses
    Sankey_data['target_bus1'] = Sankey_data['link name'].map((network.links.loc[links, 'bus1']))

    #recording the dispatch from the source bus 
    dispatch_series = (network.links_t.p0[links].sum() * network.snapshot_weightings.objective[0]).copy() #turning it into MWh
    Sankey_data['link dispatch from source [MWh/a]'] = Sankey_data['link name'].map(dispatch_series)

    #recording the arrival amount at the target bus 
    dispatch_series = (network.links_t.p1[links].sum() * network.snapshot_weightings.objective[0] * -1).copy() #turning it into MWh #multipled by negative one because power arriving at bus
    Sankey_data['link MWh arrival at target [MWh/a]'] = Sankey_data['link name'].map(dispatch_series)

    #getting carrier information 
    Sankey_data['carrier'] = Sankey_data['link name'].map(network.links.loc[links, 'carrier'])


    ##Building the Sankey Diagram 
     #aggregating sources, targets, and flows to carrier types (so I don't get one line for each country)
    to_remove = ['AL0 0', 'AT0 0', 'BA0 0', 'BE0 0', 'BG0 0',
    'CH0 0', 'CZ0 0', 'DE0 0', 'DK0 0', 'DK1 0',
    'EE0 0', 'ES0 0', 'ES6 0', 'FI1 0', 'FR0 0',
    'FR5 0', 'GB2 0', 'GB3 0', 'GR0 0', 'HR0 0',
    'HU0 0', 'IE3 0', 'IT0 0', 'IT4 0', 'LT0 0',
    'LU0 0', 'LV0 0', 'ME0 0', 'MK0 0', 'NL0 0',
    'NO1 0', 'PL0 0', 'PT0 0', 'RO0 0', 'RS0 0',
    'SE1 0', 'SI0 0', 'SK0 0', 'XK0 0']

    # Function to remove matching substrings
    def clean_value(val):
        for pattern in to_remove:
            val = val.replace(pattern, '')
        val =  val.strip()
        return val if val else 'electric network'
    #grouping the target buses by bus type 
    Sankey_data['aggregate source'] = Sankey_data['source bus'].apply(clean_value) #this is redundant because this should all be coming from one bus
    Sankey_data['aggregate targets'] = Sankey_data['target_bus1'].apply(clean_value)



        # Aggregate with carrier preserved (take first carrier in each group)
    agg_flows = Sankey_data.groupby(['aggregate source', 'aggregate targets']).agg({
        'link dispatch from source [MWh/a]': 'sum',
        'carrier': 'first'  # or another strategy if multiple carriers per link group
    }).reset_index()

    #apply source IDs and labels 
    all_labels = pd.unique(agg_flows['aggregate source'].tolist() + agg_flows['aggregate targets'].tolist())
    label_to_id = {label: i for i, label in enumerate(all_labels)}

    #for construction of the sankey diagram 

    agg_flows['source_id'] = agg_flows['aggregate source'].map(label_to_id)
    agg_flows['target_id'] = agg_flows['aggregate targets'].map(label_to_id)
     # Add custom hover text
    agg_flows['custom_text'] = (
        'Carrier: ' + agg_flows['carrier'] +
        '<br>From: ' + agg_flows['aggregate source'] +
        '<br>To: ' + agg_flows['aggregate targets'] +
        '<br>Flow: ' + agg_flows['link dispatch from source [MWh/a]'].round(2).astype(str)
    )

    # Sankey Plot
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            label=list(all_labels),
            pad=15,
            thickness=20,
            line=dict(color='black', width=0.5)
        ),
        link=dict(
            source=agg_flows['source_id'],
            target=agg_flows['target_id'],
            value=agg_flows['link dispatch from source [MWh/a]'],
            customdata=agg_flows['custom_text'],
            hovertemplate='%{customdata}<extra></extra>'
        )
    )])

    fig.update_layout(title_text="Energy Flow Sankey Diagram with Carrier Info", font_size=12)
    fig.show()

    print(Sankey_data)


In [42]:
Sankey_from_bus(network_2030, 'EU oil')

/var/folders/_w/ry357l9n0vjg8jgfczvkpk3r0000gn/T/ipykernel_44213/3033883303.py:23: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/_w/ry357l9n0vjg8jgfczvkpk3r0000gn/T/ipykernel_44213/3033883303.py:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/_w/ry357l9n0vjg8jgfczvkpk3r0000gn/T/ipykernel_44213/3033883303.py:64: FutureWarning:

unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.



                                 link name source bus  \
0                 AL0 0 land transport oil     EU oil   
1                 AT0 0 land transport oil     EU oil   
2                 BA0 0 land transport oil     EU oil   
3                 BE0 0 land transport oil     EU oil   
4                 BG0 0 land transport oil     EU oil   
..                                     ...        ...   
321  SI0 0 urban decentral oil boiler-2019     EU oil   
322            SK0 0 rural oil boiler-2015     EU oil   
323            SK0 0 rural oil boiler-2019     EU oil   
324  SK0 0 urban decentral oil boiler-2015     EU oil   
325  SK0 0 urban decentral oil boiler-2019     EU oil   

                    target_bus1  link dispatch from source [MWh/a]  \
0      AL0 0 land transport oil                       4.738796e+06   
1      AT0 0 land transport oil                       4.870263e+07   
2      BA0 0 land transport oil                       7.825273e+06   
3      BE0 0 land transport oil    

In [51]:
Sankey_from_bus(network_2030, 'EU oil')

/var/folders/_w/ry357l9n0vjg8jgfczvkpk3r0000gn/T/ipykernel_44213/3033883303.py:23: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/_w/ry357l9n0vjg8jgfczvkpk3r0000gn/T/ipykernel_44213/3033883303.py:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/_w/ry357l9n0vjg8jgfczvkpk3r0000gn/T/ipykernel_44213/3033883303.py:64: FutureWarning:

unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.



                                 link name source bus  \
0                 AL0 0 land transport oil     EU oil   
1                 AT0 0 land transport oil     EU oil   
2                 BA0 0 land transport oil     EU oil   
3                 BE0 0 land transport oil     EU oil   
4                 BG0 0 land transport oil     EU oil   
..                                     ...        ...   
321  SI0 0 urban decentral oil boiler-2019     EU oil   
322            SK0 0 rural oil boiler-2015     EU oil   
323            SK0 0 rural oil boiler-2019     EU oil   
324  SK0 0 urban decentral oil boiler-2015     EU oil   
325  SK0 0 urban decentral oil boiler-2019     EU oil   

                    target_bus1  link dispatch from source [MWh/a]  \
0      AL0 0 land transport oil                       4.738796e+06   
1      AT0 0 land transport oil                       4.870263e+07   
2      BA0 0 land transport oil                       7.825273e+06   
3      BE0 0 land transport oil    

In [5]:
import importlib
importlib.reload(af)
af.bus_connections(network_2040, 'BG0 0 land transport oil')

The loads connected to this bus are:     ['BG0 0 land transport oil']
The incoming links to this bus are from the bus:  ['EU oil']
The incoming links are:  Index(['BG0 0 land transport oil'], dtype='object', name='Link')


In [6]:
for i in [network_2020, network_2030, network_2040, network_2050]:
    network_option = i 
    land_oil_loads = network_option.loads[network_option.loads.carrier == 'land transport oil'].index #'land transport oil'
    land_transport_yearly_oil_loads = network_option.loads_t.p[land_oil_loads].sum(axis = 1).sum() * (365 * 24 ) #all hours in the year
    print(land_transport_yearly_oil_loads)

1064147468425.316
744903227897.7213
319244240527.59485
0.0


ok so EU oil is going to land transport, naptha for industry, kerosene for aviation, shipping oil, and agricultural machinery oil by 2040 and 2050 
Question 2 - what gets lumped into 'land transport oil?' 
Answer 2 - it looks like only ice car shared are actually contributing the land transport oil. 

Question 3 - are the demands for kerosene for aviation and naptha for industry variable? 
Answer 3 - see below


In [55]:
loads_changes_dataframe = pd.DataFrame()
nlist = [2020, 2030, 2040, 2050]
n=0
for i in [network_2020, network_2030, network_2040, network_2050]:
    print(str(nlist[n]) + ":")
    load_type = ['shipping oil', 'naphtha for industry','kerosene for aviation', 'agriculture machinery oil']
    network_choice = i
    relevant_loads = network_choice.loads[network_choice.loads.carrier.isin(load_type)].index
    loads_sum = network_choice.loads_t.p[relevant_loads].copy()
    loads_sum = loads_sum.T.groupby(network_choice.loads.carrier).sum().T.sum()
    print(loads_sum)
    n += 1

2020:
carrier
agriculture machinery oil    3.956582e+06
kerosene for aviation        2.861834e+07
naphtha for industry         3.965367e+07
shipping oil                 2.225182e+07
dtype: float64
2030:
carrier
agriculture machinery oil    3.956582e+06
kerosene for aviation        2.861834e+07
naphtha for industry         2.989028e+07
shipping oil                 1.557627e+07
dtype: float64
2040:
carrier
agriculture machinery oil    3.956582e+06
kerosene for aviation        2.861834e+07
naphtha for industry         1.941982e+07
shipping oil                 6.675545e+06
dtype: float64
2050:
carrier
agriculture machinery oil    3.956582e+06
kerosene for aviation        2.861834e+07
naphtha for industry         1.065089e+07
dtype: float64


the demand for kerosene for aviation doesn't change. Naptha for industry goes down, and shipping oil disapears by 2050 (after decreasing in earlier years)

In [46]:
#importlib.reload(af)
af.Sankey_biomass_hydrogen(network_2050)

/Users/katherine.shaw/Desktop/pypsa-eur/Evaluation Documents/Auxillary_Functions.py:1585: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/Users/katherine.shaw/Desktop/pypsa-eur/Evaluation Documents/Auxillary_Functions.py:1588: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/Users/katherine.shaw/Desktop/pypsa-eur/Evaluation Documents/Auxillary_Functions.py:1630: FutureWarning:

unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.



In [37]:
land_transport_buses = network_2040.buses[network_2040.buses.carrier == 'land transport oil'].index
network_2040.links[network_2040.links.bus1.isin(land_transport_buses)] #they are all just coming from EU oil directly 
#next question: what can go into EU oil? 
#network = network_2050 
#network.links[network.links.bus1 == 'EU oil']

,bus0,bus1,type,carrier,efficiency,active,build_year,lifetime,p_nom,p_nom_mod,...,dc,geometry,underwater_fraction,voltage,under_construction,underground,reversed,tags,project_status,length_original
Link,,,,,,,,,,,,,,,,,,,,,
AL0 0 land transport oil,EU oil,AL0 0 land transport oil,,land transport oil,1.0,True,0,inf,0.0,0.0,...,NaN,,NaN,NaN,NaN,NaN,False,,,0.0
AT0 0 land transport oil,EU oil,AT0 0 land transport oil,,land transport oil,1.0,True,0,inf,0.0,0.0,...,NaN,,NaN,NaN,NaN,NaN,False,,,0.0
BA0 0 land transport oil,EU oil,BA0 0 land transport oil,,land transport oil,1.0,True,0,inf,0.0,0.0,...,NaN,,NaN,NaN,NaN,NaN,False,,,0.0
BE0 0 land transport oil,EU oil,BE0 0 land transport oil,,land transport oil,1.0,True,0,inf,0.0,0.0,...,NaN,,NaN,NaN,NaN,NaN,False,,,0.0
BG0 0 land transport oil,EU oil,BG0 0 land transport oil,,land transport oil,1.0,True,0,inf,0.0,0.0,...,NaN,,NaN,NaN,NaN,NaN,False,,,0.0
CH0 0 land transport oil,EU oil,CH0 0 land transport oil,,land transport oil,1.0,True,0,inf,0.0,0.0,...,NaN,,NaN,NaN,NaN,NaN,False,,,0.0
CZ0 0 land transport oil,EU oil,CZ0 0 land transport oil,,land transport oil,1.0,True,0,inf,0.0,0.0,...,NaN,,NaN,NaN,NaN,NaN,False,,,0.0
DE0 0 land transport oil,EU oil,DE0 0 land transport oil,,land transport oil,1.0,True,0,inf,0.0,0.0,...,NaN,,NaN,NaN,NaN,NaN,False,,,0.0
DK0 0 land transport oil,EU oil,DK0 0 land transport oil,,land transport oil,1.0,True,0,inf,0.0,0.0,...,NaN,,NaN,NaN,NaN,NaN,False,,,0.0


In [39]:
#what loads are connected to land transport oil? 
af.bus_connections(network_2040, 'BG0 0 land transport oil' )

The loads connected to this bus are:     ['BG0 0 land transport oil']
The incoming links to this bus are from the bus:  ['EU oil']
The incoming links are:  Index(['BG0 0 land transport oil'], dtype='object', name='Link')
